# Deep Learning for Developers with Ludwig

<a href="https://user-images.githubusercontent.com/13848158/154338760-edfe1885-06f3-4e02-87fe-4b13a403516b.png"><img src="https://user-images.githubusercontent.com/13848158/154338760-edfe1885-06f3-4e02-87fe-4b13a403516b.png" title="source: imgur.com" /></a>

Ludwig is a toolbox built on top of TensorFlow that allows users to train and test deep learning models without the need to write code.

All you need to provide is a dataset file containing your data, a list of columns to use as inputs, and a list of columns to use as outputs, Ludwig will do the rest. Simple commands can be used to train models both locally and in a distributed way, and to use them to predict new data.

A programmatic API is also available in order to use Ludwig from your python code. A suite of visualization tools allows you to analyze models' training and test performance and to compare them.

### Why use Aim

pass

### Using Aim with Ludwig
pass

Training is easy:
`ludwig train --dataset DATA_PATH --config_file CONFIG_PATH --aim`

# Installation and demo

In [1]:
# ! pip install git+http://github.com/uber/ludwig.git -qq
# ! pip install ludwig[serve] -qq
# ! pip install aim -qq 

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

2022-05-03 23:09:39.501667: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Found GPU at: /device:GPU:0


2022-05-03 23:09:41.186360: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-03 23:09:41.192910: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2799925000 Hz
2022-05-03 23:09:41.193677: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562deac3d6b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-03 23:09:41.193694: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-05-03 23:09:41.195740: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-05-03 23:09:41.295358: I tensorflow/stream_executor/cuda/cuda_gpu_executor.

In [3]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:GPU:0']


2022-05-03 23:09:41.734378: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-03 23:09:41.734719: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2022-05-03 23:09:41.734751: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-05-03 23:09:41.734777: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-05-03 23:09:41.734793: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2022-05-03 23:09:41.734807: I tensorflow/stream_executor/platfor

### Text Classification

Text classification also known as text tagging or text categorization is the process of categorizing text into organized groups. By using Natural Language Processing (NLP), text classifiers can automatically analyze text and then assign a set of pre-defined tags or categories based on its content.

Unstructured text is everywhere, such as emails, chat conversations, websites, and social media but it’s hard to extract value from this data unless it’s organized in a certain way. Doing so used to be a difficult and expensive process since it required spending time and resources to manually sort the data or creating handcrafted rules that are difficult to maintain. 

Let's build a text classifier using ludwig.

### Kaggle's AGNews Dataset
AG is a collection of more than 1 million news articles. News articles have been gathered from more than 2000  news sources by ComeToMyHead in more than 1 year of activity. ComeToMyHead is an academic news search engine which has been running since July, 2004. The dataset is provided by the academic comunity for research purposes in data mining (clustering, classification, etc), information retrieval (ranking, search, etc), xml, data compression, data streaming, and any other non-commercial activity. For more information, please refer to the link http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html .

The articles are divided into 4 classes:
```
World
Sports
Business
Sci/Tech
```
Let's download the dataset. The dataset from kaggle has been pre-processed and uploaded to W&B as dataset artifact. It can be downloaded using the API that comes associated with each artifact.

In [4]:
import aim

In [5]:
id_to_label = {
   1: 'World', 2: 'Sports', 3: 'Business', 4: 'Sci/Tech'
    }

In [6]:
# import pandas as pd

# train_csv = pd.read_csv("train.csv")
# train_csv.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [26]:
# train_csv[:100]

,ClassIndex,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
95,4,Gene Blocker Turns Monkeys Into Workaholics - ...,Reuters - Procrastinating monkeys were turned\...
96,4,Dolphins Too Have Born Socialites (Reuters),Reuters - Some people are born to be the life ...
97,4,"What's in a Name? Well, Matt Is Sexier Than Pa...","Reuters - As Shakespeare said, a rose by any o..."
98,4,UK Scientists Allowed to Clone Human Embryos (...,Reuters - British scientists said on Wednesday...


In [39]:
# train_csv = train_csv.rename(columns={"Class Index":"ClassIndex"})
# train_csv[:300].to_csv("final_train.csv")
# train_csv.head()


,ClassIndex,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


## Experiment Tracking

pass

## Train 
This command lets you train a model from your data. You can call it with:


In [55]:
!ludwig train --dataset final_train.csv --config train_conf.yaml -g 0 \
--aim --experiment_name "Classification"


NumExpr defaulting to 8 threads.
2022-05-05 21:29:52.564959: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/home/erik/anaconda3/envs/ludwig/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
ray.init() failed: Could not find any running Ray instance. Please specify the one to connect to by setting `--address` flag or `RAY_ADDRESS` environment variable.
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.5rc2 - Train


╒════════════════════════╕
│ EXPERIMENT DESCRIPTI


Dataset sizes:
╒════════════╤════════╕
│ Dataset    │   Size │
╞════════════╪════════╡
│ Training   │    209 │
├────────────┼────────┤
│ Validation │     28 │
├────────────┼────────┤
│ Test       │     63 │
╘════════════╧════════╛
aim.on_train_init() called...
base config  {'input_features': [{'name': 'Title', 'type': 'text'}, {'name': 'Description', 'type': 'text'}], 'output_features': [{'name': 'ClassIndex', 'type': 'category'}], 'trainer': {'epochs': 2}}
experiment directory  /home/erik/Documents/UCPH/aim_projs/ludwig/aim_demo/results/Classification_run_21
experiment name  Classification
model name  run
output directory  /home/erik/Documents/UCPH/aim_projs/ludwig/aim_demo/results/Classification_run_21
{'name': 'run', 'dir': '/home/erik/Documents/UCPH/aim_projs/ludwig/aim_demo/results/Classification_run_21'}

╒═══════╕
│ MODEL │
╘═══════╛

Warnings and other logs:
/home/erik/anaconda3/envs/ludwig/lib/python3.8/site-packages/torch/nn/modules/conv.py:298: UserWarning: Using padding='s

START OF BATCH
{'batch_size': 128, 'epoch': 1, 'steps': 2, 'tune_checkpoint_num': 0, 'last_improvement_steps': 2, 'learning_rate': 0.001, 'best_valid_metric': 0.7264037728309631, 'num_reductions_lr': 0, 'num_increases_bs': 0, 'train_metrics.ClassIndex.loss': 0.6891297101974487, 'train_metrics.ClassIndex.accuracy': 0.7416267991065979, 'train_metrics.combined.loss': 0.6891297101974487, 'validation_metrics.ClassIndex.loss': 0.7264037728309631, 'validation_metrics.ClassIndex.accuracy': 0.7142857313156128, 'validation_metrics.combined.loss': 0.7264037728309631, 'test_metrics.ClassIndex.loss': 0.6979712247848511, 'test_metrics.ClassIndex.accuracy': 0.7460317611694336, 'test_metrics.combined.loss': 0.6979712247848511}
_________________
Training:  75%|██████████████████████████▎        | 3/4 [00:03<00:01,  1.41s/it]END OF BATCH
{'batch_size': 128, 'epoch': 1, 'steps': 3, 'tune_checkpoint_num': 0, 'last_improvement_steps': 2, 'learning_rate': 0.001, 'best_valid_metric': 0.7264037728309631, 'num

You get all of these detailed insights about the training process in the Aim dashboard:



## Predict
This command lets you use a previously trained model to predict on new data. You can call it with:

In [ ]:
!ludwig predict --dataset final_test.csv \
--model_path results/Classification_run/model

On performing prediction, you get the following files in both csv and npy format:
* Class Index predictions 
* Class Index probalities for each class
* Highest Class Index probability

In [ ]:
prediction = pd.read_csv('results/ClassIndex_predictions.csv')

In [ ]:
# Check on some random examples
test_dataset = pd.read_csv('final_test.csv')
index = [100,900,575,1100,1500]
for i in index:
  print(test_dataset.iloc[i], '\n Prediction -> ', id_to_label[prediction.iloc[i][0]], '\n')


## Evaluate
This command lets you use a previously trained model to predict on new data and evaluate the performance of the prediction compared to ground truth. You can call it with:




In [ ]:
!ludwig evaluate --dataset final_test.csv \
--model_path results/Classification_run/model

Running evaluation saves the evaluation metrics in the results folder in json format

# Visualize
Ludwig comes with many visualization options. If you want to look at the learning curves of your model for instance, run:

In [ ]:
!ludwig visualize \
--visualization learning_curves \
--training_statistics results/Classification_run/training_statistics.json \
--output_directory results/

using `--output_directory ` argument saves the outputs in the desired directory instead of directly displaying them.

## Serving
This command lets you load a pre-trained model and serve it on an http server. It uses port 8000 by default.
Once the server is up and running, you can pass the parameters defined the model configuration as inputs. 
Example:
```
curl http://0.0.0.0:8000/predict -X POST -F 'Title=Science' -F 'Description=Techology'
```

In [ ]:
!ludwig serve --model_path results/Classification_run/model

### Load the saved artifact
The saved artifacts can be accessed using the API that automatically gets generated once you upload the artifact. You can then perform transfer learning, prediction or even deploy the models downloaded using artifacts.

# Other Features
There are many other useful commands supported by Ludwig CLI such as:
```
hyperopt: Perform hyperparameter optimization
collect_summary: Prints names of weights and layers activations to use with other collect commands
collect_weights: Collects tensors containing a pretrained model weights
collect_activations: Collects tensors for each datapoint using a pretrained model
export_savedmodel: Exports Ludwig models to SavedModel
export_neuropod: Exports Ludwig models to Neuropod
preprocess: Preprocess data and saves it into HDF5 and JSON format
synthesize_dataset: Creates synthetic data for tesing purposes
```
To know more about these features please visit the [official Ludwig docs](https://ludwig-ai.github.io/ludwig-docs/user_guide/) 